<a href="https://colab.research.google.com/github/daniballester-ai/Tradutor_artigos_Azure_AI/blob/main/Tradutor_Artigos_Tecnicos_AzureAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traduzindo documentos

In [1]:
!pip install requests python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 5.4 MB/s eta 0:00:00


In [8]:
import requests
from docx import Document
import os

subscription_key = "6b29fc40ca4740f5a0c0b1a03a0a3822"
endpoint = 'https://api.cognitive.microsofttranslator.com'
location = 'eastus2'
language_destination = 'pt-br'

def translator_text(text, target_language):
    path = '/translate'
    constructed_url = endpoint + path
    headers = {
        'Ocp-Apim-Subscription-Key': subscription_key,
        'Ocp-Apim-Subscription-Region': location,
        'Content-type': 'application/json',
        'X-ClientTraceId  ': str(os.urandom(16))
    }

    body = [{
        'text': text
    }]

    params = {
        'api-version': '3.0',
        'from': 'en',
        'to': target_language
    }

    request = requests.post(constructed_url, headers=headers, json=body)
    response = request.json()

    return response[0]['translations'][0]['text']

In [11]:
#Chamando a função de tradução de documento
translator_text("I know you're somewhere out there, somewhere far away", language_destination)

'Eu sei que você está em algum lugar lá fora, em algum lugar distante'


In [14]:
def translate_document(path):
  document = Document(path)
  full_text = []
  for paragraph in document.paragraphs:
    translated_text = translator_text(paragraph.text, language_destination)
    full_text.append(translated_text)

  translated_doc = Document()
  for line in full_text:
    translated_doc.add_paragraph(line)

  path_translated = path.replace('.docx', f"_(language_destination).docx")
  translated_doc.save(path_translated)
  return path_translated

input_file = "/content/MUSICA.docx"
translate_document(input_file)

Eu sei que você está em algum lugar por aí, em algum lugar bem longe
Eu quero você de volta, eu quero você de volta
Meus vizinhos acham que eu sou louco, mas eles não entendem
Você era tudo o que eu tinha, você era tudo o que eu tinha

À noite, quando as estrelas iluminam meu quarto
Eu me sento sozinho
Conversando com a lua
Tentando chegar até você
Na esperança de que você esteja do outro lado falando comigo também
Ou será que sou um tolo que fica sozinho conversando com a lua?
Oh

Estou me sentindo como se eu fosse famoso, o assunto da cidade
Eles dizem que eu enlouqueci, sim, eu enlouqueci
Mas eles não sabem o que eu sei, porque quando o sol se põe
Alguém está respondendo, sim, estão me respondendo, oh

À noite, quando as estrelas iluminam meu quarto
Eu me sento sozinho
Conversando com a lua
Tentando chegar até você
Na esperança de que você esteja do outro lado falando comigo também
Ou será que sou um tolo que fica sozinho conversando com a lua?
Ah, ah, ah

Você consegue me ouvir cha

# Traduzindo Artigos

In [15]:
!pip install requests beautifulsoup4 openai langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.75
    Uninstalling langchain-core-0.3.75:
      Successfully uninstalled langchain-core-0.3.75


In [17]:
import requests
from bs4 import BeautifulSoup

def extract_text_from_url(url):
  response = requests.get(url)

  if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    for script_or_style in soup(["script", "style"]):
      script_or_style.decompose()
    text = soup.get_text(separator=' ')
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return text
  else:
    print(f"Failed to fetch the URL. Status code: {response.status_code}")
    return None

  text = soup.get_text()
  return text

extract_text_from_url('https://dev.to/shakenbeer/i-just-had-a-revelation-17cb')

"I just had a revelation - DEV Community\nForem Feed\nFollow new Subforems to improve your feed\nDEV Community\nFollow\nA space to discuss and keep up software development and manage your software career\nGamers Forem\nFollow\nAn inclusive community for gaming enthusiasts\nFuture\nFollow\nNews and discussion of science and technology such as AI, VR, cryptocurrency, quantum computing, and more.\nMusic Forem\nFollow\nFrom composing and gigging to gear, hot music takes, and everything in between.\nDUMB DEV Community\nFollow\nMemes and software development shitposting\nVibe Coding Forem\nFollow\nDiscussing AI software development, and showing off what we're building.\nPopcorn Movies and TV\nFollow\nMovie and TV enthusiasm, criticism and everything in-between.\nDesign Community\nFollow\nWeb design, graphic design and everything in-between\nMaker Forem\nFollow\nA community for makers, hobbyists, and professionals to discuss Arduino, Raspberry Pi, 3D printing, and much more.\nScale Forem\nFol

In [31]:
from langchain_openai.chat_models.azure import AzureChatOpenAI

client = AzureChatOpenAI(
    azure_endpoint="https://oai-dio-bootcamp-dev-eastus-001.openai.azure.com/",
    api_key="730f2276a6a54c01964cf11672f08e7c",
    api_version="2024-02-15-preview",
    deployment_name="gpt-4o-mini",
    max_retries=0
)

def translate_article(text, lang):
  messages = [
      ("system", "Você atua como tradutor de textos"),
      ("user", f"Traduza o  {text} para o idioma {lang} e responda em markdown")
  ]
  response = client.invoke(messages)
  return response.content

translate_article("A space to discuss and keep up software development and manage your software career", "pt-br")

'Um espaço para discutir, se manter atualizado sobre desenvolvimento de software e gerenciar sua carreira'


In [33]:
url = 'https://dev.to/shakenbeer/i-just-had-a-revelation-17cb'
text = extract_text_from_url(url)
article = translate_article(text, "pt-br")
print(article)

**Eu acabei de ter uma revelação**

`#kmp` `#android`

---

**KMP para iOS não está pronto para produção.**

Na minha opinião, é tudo por dinheiro. O objetivo é criar um novo produto e promovê-lo da forma mais rápida e agressiva possível. Isso muitas vezes significa mentir em nossas propagandas e nunca parar o ciclo. Se nós parássemos, os desenvolvedores talvez percebessem que não havia nada de errado com `Java` e as views em `XML`.

Sem a pressão para acompanhar as novas tecnologias do `Android`, os desenvolvedores `Android` poderiam aprender a desenvolver para `iOS`. Eventualmente, eles poderiam perceber que uma base de código compartilhada e multiplataforma não precisa de `Kotlin` ou um novo framework quando `C` e `C++` já estão disponíveis.
